In [ ]:
import sys
import numpy as np
import math
import matplotlib.pyplot as plt
import os
import scipy.io
import numpy as np
import scipy as sp
import pickle
import time
from operator import itemgetter
import statistics as st
import matplotlib
import random
matplotlib.rcParams['figure.dpi'] = 500


############################# FUNCTIONS ################################################################
########################################################################################################
########################################################################################################
########################################################################################################

# check whether we are fitting constrained elements of covariance matrix within desired tolerance
# inputs: model covariance matrix (J_inv), data covariance matrix (cov_mat), current model adjacency matrix (adj_mat), 
#         error tolerance for constrained elements (tol)
# outputs: True if all constrained elements are within tolerances, False if not
def check_correct(J_inverse,cov_mat,adj_mat,tol):
    diff = np.multiply(adj_mat,abs(J_inverse-cov_mat))
    avg = np.mean(diff)
    max_val = np.amax(diff)
    if max_val>tol:
        return False
    else:
        return True

# check whether we are fitting constrained elements of precision matrix within desired tolerance
# inputs: model precision matrix (J), current model adjacency matrix (adj_mat), error tolerance for constrained elements (tol)
# outputs: True if all constrained elements are within tolerances, False if not
def check_correct_J(J,adj_mat,tol):
    adj_mirror = np.ones((N,N)) - adj_mat
    diff = np.multiply(adj_mirror,abs(J))
    avg = np.mean(diff)
    max_val = np.amax(diff)
    if max_val>tol:
        return False
    else:
        return True

# calculate the entropy of a gaussian
# input: covariance matrix (mat)
# output: entropy (ent)
def entropy(mat):
    eigvals = np.linalg.eigvals(mat)
    exponents = -np.log(eigvals)
    log_det = -sum(exponents) 
    ent = 0.5*N*np.log(2.0*math.pi*math.e) + 0.5*log_det
    return ent

# find maxent model with given constraints
# this is algorithm 2 from the Uhler paper
# inputs: adjacency matrix (adj_mat), data covariance matrix (cov_mat), desired accuracy/tolerance (tol)
# returns: model covariance matrix (C)
def maxent(adj_mat,cov_mat,tol):
    ent_list_maxent = []   
    tolerance = tol
    K = np.identity(N)
    C = np.identity(N)
    K_last_step = np.zeros((N,N))
    I2 = np.eye(2)
    steps = 0
    while check_correct(C,cov_mat,adj_mat,tolerance) == False: 
        for i in range(0,len(adj_mat)):
            for j in range(i,len(adj_mat)): 
                if adj_mat[i,j]==1:
                    if i!=j:
                        a = [i,j]
                        b = np.setdiff1d(list(range(0,N)),a)
                        
                        Saa = cov_mat.take(a,axis = 0).take(a,axis = 1)
                        Kab = K.take(a,axis = 0).take(b,axis = 1)
                        Kba = K.take(b,axis = 0).take(a,axis = 1)
                        
                        Cab = C.take(a,axis = 0).take(b,axis = 1)
                        Cba = C.take(b,axis = 0).take(a,axis = 1)
                        Cbb = C.take(b,axis = 0).take(b,axis = 1)
                        Caa = C.take(a,axis = 0).take(a,axis = 1)
                        Caa_inv = np.linalg.inv(Caa)
                        Kbb_inv = Cbb - np.linalg.multi_dot([Cba,Caa_inv,Cab])
                        
                        Kaa_new = np.linalg.inv(Saa) + np.linalg.multi_dot([Kab,Kbb_inv,Kba])

                        K_update = np.array(Kaa_new) - K[i:j+1:j-i,i:j+1:j-i]

                        K[i:j+1:j-i,i:j+1:j-i] = np.array(Kaa_new)
    
                        # calculate new J here:
                        U = np.zeros((N,2))
                        V = np.zeros((2,N))

                        U[i,0]= K_update[0,0]
                        U[j,1]= K_update[1,1]
                        U[i,1]= K_update[0,1]
                        U[j,0]= K_update[1,0]
                        V[0,i]=1
                        V[1,j]=1
    
                        C = C - np.linalg.multi_dot([C,U,np.linalg.inv(I2+np.linalg.multi_dot([V,C,U])),V,C])

                    else:
                        a = [i]
                        b = np.setdiff1d(list(range(0,N)),a)

                        Kab = K.take(a,axis = 0).take(b,axis = 1)
                        Kba = K.take(b,axis = 0).take(a,axis = 1)
                        Kbb = K.take(b,axis = 0).take(b,axis = 1)
                        Cab = C.take(a,axis = 0).take(b,axis = 1)
                        Cba = C.take(b,axis = 0).take(a,axis = 1)
                        Cbb = C.take(b,axis = 0).take(b,axis = 1)
                        Caa = C.take(a,axis = 0).take(a,axis = 1)
                        Caa_inv = np.linalg.inv(Caa)
                        Kbb_inv = Cbb - np.linalg.multi_dot([Cba,Caa_inv,Cab])
                        
                        Kaa_new = 1.0/cov_mat[i,i] + np.linalg.multi_dot([Kab,Kbb_inv,Kba])
                        K_update = np.array(Kaa_new) - K[i,i]
                        K[i,i] = Kaa_new
    
                        # calculate new J here:
                        U = np.zeros((N,1))
                        V = np.zeros((1,N))

                        U[i,0]= K_update
                        V[0,i]=1
                        C = C - np.linalg.multi_dot([C,U,V,C])/(1.0+np.linalg.multi_dot([V,C,U]))

        steps = steps + 1

    return C


# find maxent model with given constraints
# this is algo 1 from the Uhler paper, the one that works better when many edges have been added
# inputs: adjacency matrix (adj_mat), data covariance matrix (cov_mat), desired accuracy/tolerance (tol)
# returns: model covariance matrix (C)
def other_maxent(adj_mat,cov_mat,tol):
    ent_list_maxent = []   
    tolerance = tol
    C = cov.copy()
    C_last_step = np.zeros((N,N))
    steps = 0
    J = np.linalg.inv(C)
    I2 = np.eye(2)

    while check_correct_J(J,adj_mat,tolerance) == False: 
        C_last_step = C.copy()
        for i in range(0,len(adj_mat)):
            for j in range(i,len(adj_mat)): 
                # now we want the ones that aren't included
                if adj_mat[i,j]==0 and i!=j:
                    a = [i,j]
                    b = np.setdiff1d(list(range(0,N)),a)
                    
                    Cib = C.take([i],axis = 0).take(b,axis = 1)
                    Cbj = C.take(b,axis = 0).take([j],axis = 1)

                    Jab = J.take(a,axis = 0).take(b,axis = 1)
                    Jba = J.take(b,axis = 0).take(a,axis = 1)
                    Jbb = J.take(b,axis = 0).take(b,axis = 1)
                    Jaa = J.take(a,axis = 0).take(a,axis = 1)
                    Jaa_inv = np.linalg.inv(Jaa)

                    Cbb_inv = Jbb - np.linalg.multi_dot([Jba,Jaa_inv,Jab])

                    Cij_new = np.linalg.multi_dot([Cib,Cbb_inv,Cbj])
                    C_update = Cij_new[0] - C[i,j]

                    C[i,j] = Cij_new[0]
                    C[j,i] = Cij_new[0]

                    # calculate new J here:
                    U = np.zeros((N,2))
                    V = np.zeros((2,N))
                    U[i,0]=C_update
                    U[j,1]=C_update
                    V[0,j]=1
                    V[1,i]=1

                    J = J - np.linalg.multi_dot([J,U,np.linalg.inv(I2+np.linalg.multi_dot([V,J,U])),V,J])
                    
        steps = steps + 1

    return C  

# This algorithm builds the models by adding more and more correlations, always picking the largest correlations in magnitude
# that haven't yet been constrained
# inputs: model covariance matrix (cov), desired accuracy/tolerance on constrained correlations for maxent algo 2 (tol), 
#         desired accuracy/tolerance on constrained correlations for maxent algo 2 (J_tol), 
#         number of edges we want added when we stop (z_target), 
# outputs: matrix of model adjacency matrices, one for each step (mat_of_adjs). NOTE - includes diagonal elements from the start
#          matrix of model covariance matrices, one for each step (mat_of_J_invs)
#          list of edges added (edge_list) - NOTE, diagonal elements (self connections) don't get added to this list until the start 
#                                            of run_minimax_heuristic. This is just a quirk with how it was written, the self-connections 
#                                            are really included from the start
#          list of numbers of correlations added (x_axis_list)
#          list of model entropies (ent_list)
def run_minimax_LCs(cov,tol,J_tol,z_target):
    
    N_parcels = 100
    
    N = 100
    adjacency = np.identity(N_parcels)
    J_inv = np.identity(N_parcels)

    mat_of_adjs = []
    mat_of_adjs.append(adjacency)
    mat_of_J_invs = []
    
    ent_list = []
    J_inv = maxent(adjacency,cov,tol)
    
    mat_of_J_invs.append(J_inv)
    ent = entropy(J_inv)
    ent_list.append(ent)
    
    x_axis_list = []
    x_axis_list.append(0)
    
    edge_list = []
    
    edge_list_backup = edge_list.copy()

    J_inv = mat_of_J_invs[-1].copy()
    adj = mat_of_adjs[-1].copy()

    # taking the absolute value here is what gives us rank by magnitude
    cov_copy = np.abs(cov.copy())
    for i in range(N):
        cov_copy[i,i] = float('-inf')

    J = np.linalg.inv(J_inv)
    
    z=0
    while z <= z_target:
        
        increment = 0
        if z<200: increment = 1
        elif z<400: increment = 2
        elif z<600: increment = 5
        elif z<800: increment = 10
        elif z<1000: increment = 20
        else: increment = 50
        
        for i in range(increment):
            max_val_loc = np.unravel_index(np.argmax(cov_copy),(100,100))
            max_val = cov_copy[max_val_loc[0],max_val_loc[1]]
            # By setting the corresponding elements to minus infinity we ensure they won't be added again
            cov_copy[max_val_loc[0],max_val_loc[1]] = float('-inf')
            cov_copy[max_val_loc[1],max_val_loc[0]] = float('-inf')
            adj[max_val_loc[0],max_val_loc[1]] = 1
            adj[max_val_loc[1],max_val_loc[0]] = 1
            edge_list.append((max_val_loc[0],max_val_loc[1]))
        
        if z<1500: 
            J_inv = maxent(adj,cov,tol)
        else: 
            J_inv = other_maxent(adj,cov,J_tol)
        z = z+increment
        x_axis_list.append(z)
        
        mat_of_J_invs.append(J_inv.copy())
        mat_of_adjs.append(adj.copy())
        ent = entropy(J_inv)
        ent_list.append(ent)


        if z%50==0:
            plt.plot(x_axis_list,(ent_list - entropy(cov))/(entropy(cov_ind) - entropy(cov)))
            plt.ylim(0,1)
            plt.xlim(0,4950)
            plt.savefig(out_path + "_ent_fig.png")
            plt.show()
    return mat_of_adjs,mat_of_J_invs,edge_list,x_axis_list,ent_list


########################################################################################################
########################################################################################################
########################################################################################################
########################################################################################################
########################################################################################################

N = 100

cov_path = # enter path to the covariance matrix (stored with pickle)
out_path = # enter path to place where you want to store output
os.mkdir(#make the directory where you wish to store outputs)

# load the covariance
f = open(cov_path + r"cov",'rb')
cov = pickle.load(f)
f.close()

# get independent model covariance (used to calculate independent entropy)
cov_ind = np.diag(np.diagonal(cov))

# run minimax, picking strongest correlations
mat_of_adjs,mat_of_J_invs,edge_list,x_axis_list,ent_list = run_minimax_LCs(cov,tol,J_tol,4950)

# make plot of scaled entropy vs fraction of edges added
plt.plot(x_axis_list,(ent_list - entropy(cov))/(entropy(cov_ind) - entropy(cov)))
plt.savefig(out_path + "_ent_fig.png")

# save outputs
f = open(out_path + "_mat_of_adjs",'wb')
pickle.dump(mat_of_adjs,f)
f.close()

f = open(out_path + "_mat_of_J_invs",'wb')
pickle.dump(mat_of_J_invs,f)
f.close()

# NOTE A QUIRK WITH THE EDGE LIST - THE DIAGONAL ENTRIES AREN'T ADDED TO THIS LIST, BUT THIS DOES NOT AFFECT 
# THE RUNNING OF ANY ALGORITHM OR ANY OUTPUTS, AND THE DIAGONAL ENTRIES WERE ALREADY CONSTRAINED FROM THE START
f = open(out_path + "_edge_list",'wb')
pickle.dump(edge_list,f)
f.close()

f = open(out_path + "_x_axis_list",'wb')
pickle.dump(x_axis_list,f)
f.close()

f = open(out_path + "_ent_list",'wb')
pickle.dump(ent_list,f)
f.close()